In [18]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import pandas as pd

### 데이터 로드 pd.read_csv(' ').values

In [2]:
# Load Kaggle Data as matrix
train_csv_data = pd.read_csv('./titanic_train.csv').values
test_csv_data = pd.read_csv('./titanic_test.csv').values
test_csv_sub = pd.read_csv('./titanic_gender_submission.csv').values

print('train_csv_data.shape = ', train_csv_data.shape, ', type(train_csv_data) = ', type(train_csv_data))
print('test_csv_data.shape = ', test_csv_data.shape, ', type(test_csv_data) = ', type(test_csv_data))
print('test_csv_sub.shape = ', test_csv_sub.shape, ', type(test_csv_sub) = ', type(test_csv_sub))

train_csv_data.shape =  (891, 12) , type(train_csv_data) =  <class 'numpy.ndarray'>
test_csv_data.shape =  (418, 11) , type(test_csv_data) =  <class 'numpy.ndarray'>
test_csv_sub.shape =  (418, 2) , type(test_csv_sub) =  <class 'numpy.ndarray'>


### 데이터 변환 (문자 -> 숫자)

In [3]:
# train data 에서 male -> 1.0, female -> 0.0
for i in range(len(train_csv_data)):
    if train_csv_data[i, 4] == 'male':
        train_csv_data[i, 4] = 1.0
    else:
        train_csv_data[i, 4] = 0.0
        
# test data 에서 male -> 1.0, female -> 0.0
for i in range(len(test_csv_data)):
    if test_csv_data[i, 3] == 'male':
        test_csv_data[i, 3] = 1.0
    else:
        test_csv_data[i, 3] = 0.0

In [4]:
# train data 에서 Embarked, Empty -> 0.0, S -> 1.0, C -> 2.0, Q -> 3.0
for i in range(len(train_csv_data)):
    if train_csv_data[i, 11] == 'S': 
        train_csv_data[i, 11] = 1.0   
    elif train_csv_data[i, 11] == 'C':     
        train_csv_data[i, 11] = 2.0     
    elif train_csv_data[i, 11] == 'Q':    
        train_csv_data[i, 11] = 3.0    
        
    if np.isnan(train_csv_data[i, 11]):  
        train_csv_data[i, 11] = 0.0
        

# test data 에서 Embarked, Empty -> 0.0, S -> 1.0, C -> 2.0, Q -> 3.0
for i in range(len(test_csv_data)):
    if test_csv_data[i, 10] == 'S':    
        test_csv_data[i, 10] = 1.0    
    elif test_csv_data[i, 10] == 'C':   
        test_csv_data[i, 10] = 2.0     
    elif test_csv_data[i, 10] == 'Q':   
        test_csv_data[i, 10] = 3.0
        
    if np.isnan(test_csv_data[i, 10]):    
        test_csv_data[i, 10] = 0.0

### test_data / training_data 생성

In [5]:
training_x_data = train_csv_data[ : , [ 2, 4, 6, 7, 11 ] ]  # Pclass, Sex, SibSp, Parch, Embarked
training_t_data = train_csv_data[ :, 1:2]

test_x_data = test_csv_data[ : , [ 1, 3, 5, 6, 10 ] ]  # Pclass, Sex, SibSp, Parch, Embarked
test_t_data = test_csv_sub[ :, 1:2]  # test_csv_sub 에서 데이터 가져옴. 주의 요함

print("training_x_data.shape =", training_x_data.shape, ", training_t_data.shape =", training_t_data.shape)
print("test_x_data.shape =", test_x_data.shape, ", test_t_data.shape =", test_t_data.shape)

training_x_data.shape = (891, 5) , training_t_data.shape = (891, 1)
test_x_data.shape = (418, 5) , test_t_data.shape = (418, 1)


### TensorFlow - 노드 / 연산 정의

In [12]:
# Hyper-Parameter Definition
learning_rate = 1e-2
input_nodes = training_x_data.shape[1]  
hidden_1_nodes = 100
output_nodes = 1
epochs = 1

In [13]:
# 입력과 출력을 위한 플레이스홀더 정의
X_DATA = tf.placeholder(tf.float32, [None, input_nodes])  
T_DATA = tf.placeholder(tf.float32, [None, output_nodes])  

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_1_nodes]))  # 은닉층 1가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_1_nodes]))               # 은닉층 1바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_1_nodes, output_nodes])) # 출력층 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))               # 출력층 바이어스 노드

In [14]:
# feed forward
Z2 = tf.matmul(X_DATA, W2) + b2
A2 = tf.sigmoid(Z2)

Z3 = tf.matmul(A2, W3) + b3  
y = A3 = tf.sigmoid(Z3)    # 최종 계산값

# 손실함수는 Cross-Entropy 
loss = -tf.reduce_mean( T_DATA*tf.log(y) + (1-T_DATA)*tf.log(1-y) ) 

In [15]:
# GradientDescent 
optimizer = tf.train.GradientDescentOptimizer(learning_rate)  # 앞으로 이 부분만 바뀔 것임

train = optimizer.minimize(loss)

In [16]:
# 정확성 검사, True if y > 0.5 else False
predicted = tf.cast(y > 0.5, dtype=tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T_DATA), dtype=tf.float32))

### TensorFlow - 노드 / 연산 실행

In [20]:
with tf.Session() as sess:  # with를 쓰면 close 하지 않아도 됨
    sess.run(tf.global_variables_initializer())  # 변수 노드 (tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):
        for step in range(10001):
            loss_val, _ = sess.run([loss, train], feed_dict={X_DATA: training_x_data, T_DATA: training_t_data})    
        
            if step % 1000 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now()
    
    print("")
    print("Elapsed Time => ", end_time-start_time)
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X_DATA: test_x_data, T_DATA: test_t_data})
    
    print("\ny_val.shape", y_val.shape, ", predicted_val =", predicted_val.shape)
    print("\nAccuracy =", accuracy_val)

epochs =  0 , step =  0 , loss_val =  1.4571259
epochs =  0 , step =  1000 , loss_val =  0.4562624
epochs =  0 , step =  2000 , loss_val =  0.44368717
epochs =  0 , step =  3000 , loss_val =  0.43834534
epochs =  0 , step =  4000 , loss_val =  0.43500233
epochs =  0 , step =  5000 , loss_val =  0.43253902
epochs =  0 , step =  6000 , loss_val =  0.43058613
epochs =  0 , step =  7000 , loss_val =  0.4289819
epochs =  0 , step =  8000 , loss_val =  0.4276364
epochs =  0 , step =  9000 , loss_val =  0.42649135
epochs =  0 , step =  10000 , loss_val =  0.4255049

Elapsed Time =>  0:00:09.434013

y_val.shape (418, 1) , predicted_val = (418, 1)

Accuracy = 0.9521531
